In [1]:
import pickle
import pandas as pd
import re
import Algorithmia
import bs4 as bs
import lxml
from urllib.request import Request, urlopen
import json
import time

In [2]:
with open('news.pkl', 'rb') as f:
    news = pickle.load(f)
    # 3863 unique authors

muckrack = pd.read_csv('muckrack_persons_fetchlist.csv')

In [3]:
client = Algorithmia.client('simKLMHlWDMzWd+QcNRqVhI104r1')
algo = client.algo('specrom/Google_scraper/0.1.4')
algo.set_options(timeout=300) # optional

In [4]:
def get_top(n):
    temp = news[['author', 'article_count']].groupby('author').count().sort_values(['article_count'], ascending = False)
    temp = temp.reset_index().head(n)
    urls = []
    for i in range(len(temp)):
        url = temp.loc[i].author.lower()
        url = re.sub(r'\|.*$', '', url) # remove everything after |
        url = re.sub(r'\,.*$', '', url) # remove evrything after ,
        
        punctuations = '''!()[]{};:'"\,<>./?#$%^&*_~'''
        for x in url.lower(): 
            if x in punctuations: url = url.replace(x, "") # remove punctuations except hyphen
        url = re.sub(r'(\d+)', '', url) # remove numbers
        url = re.sub(r'(--)', '', url)
        url = ' '.join([i.strip() for i in url.split()]) # remove spaces and lowercase
        url = re.sub(r'(\w*@\w*)', '', url) # remove emails
        
        url = re.sub(r'(sa)(\s+)(\w+)(\s+)', '', url) # SA editors
        url = re.sub(r'(tulsa)(\s+)(world)', '', url) # tulsa world editors
        url = re.sub(r'(world-herald)(\s*)(\w*)(\s*)(\w*)', '', url) # world herald editors
        url = re.sub(r'(news)(\s+)(editor)', '', url) # news editors
        url = re.sub(r'(richmond)(\s+)(times-dispatch)', '', url) # richmond editors
        url = re.sub(r'(new)(\s+)(hampshire)(\s+)(union)(\s+)(leader)', '', url) # new hampshire editors
        url = re.sub(r'(for)(\s+)(the)(\s+)(state)(\s+)(journal)', '', url) # state journal editors
        url = re.sub(r'(union)(\s+)(leader)(\s+)(correspondent)', '', url) # union leader correspondants
        url = re.sub(r'(sfgate)', '', url) # sfgate correspondants
        url = re.sub(r'(special)(\s+)(from)(\s+)(the)(\s+)(gazette)', '', url) # the gazette correspondants
        url = re.sub(r'(times)(\s+)(correspondent)', '', url) # times correspondants
        
        url = 'https://muckrack.com/' + url.replace(' ', '-').strip('-') # make URL
        urls.append(url)
    temp['Muckrack'] = urls
    return temp

In [5]:
data = get_top(10)

In [6]:
def extract_data(soup):
    info = {}
    info['name'] = data.iloc[j].author
    if soup.find_all(class_ = 'profile-verified'):
        info['verified'] = True
    if soup.find(class_ = 'person-details-item person-details-beats'):
        info['interests'] = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-tasks icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    if soup.find(class_ = 'person-details-item person-details-title'):
        info['title'] = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-building icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    if soup.find(class_ = 'person-details-item person-details-location'):
        info['location'] = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-map-marker icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    if soup.find(class_ = 'mr-font-size-lg mr-font-family-2 top-sm'):
        info['description'] = soup.find(class_ = 'mr-font-size-lg mr-font-family-2 top-sm').text.strip()
    info['seen_in'] = list(set([i.text for i in soup.find_all('a') if '/media-outlet/' in i.attrs['href']]))
    if soup.find(class_ = 'profile-section-social'):
        for i in soup.find(class_ = 'profile-section-social'):
            temp = [j[5:] for j in str(i).split() if 'href' in j]
            if temp: 
                temp = temp[0].strip('"')
                if 't.co' in temp:
                    info['website'] = temp
                if 'twitter' in temp:
                    info['twitter'] = temp
                if 'linkedin' in temp:
                    info['linkedin'] = temp
                if 'facebook' in temp:
                    info['facebook'] = temp
                if 'instagram' in temp:
                    info['instagram'] = temp      
    print(info)

In [7]:
for j in range(len(data)):
    try:
        url = data.iloc[j].Muckrack
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        sauce = urlopen(req).read()
    except:
        print("wrong url", j, data.iloc[j].author)
        query = {"query": f"{data.iloc[j].author} site: muckrack"}
        url = algo.pipe(query).result
        url = url[0]['url']
        if 'muckrack' in url:    
            print("right url", url)
            print()
            req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            sauce = urlopen(req).read()
    soup = bs.BeautifulSoup(sauce,'lxml')
    extract_data(soup)
    print()

{'name': 'Neil Shaw', 'verified': True, 'interests': 'United Kingdom', 'title': 'Network Content Editor — DevonLive, Reach plc', 'location': 'England', 'description': 'Network Content Editor for Reach Plc. All of my words are second-hand, and useless in the face of this.', 'seen_in': ['The Washington Post', 'Reach plc', 'Al Jazeera', 'Daily Star (UK)', 'MSN South Africa', 'Fox News', 'Associated Press', 'DevonLive', 'Daily Mirror', 'The (Toronto) Star', 'MSN UK', 'Sunday Mirror'], 'website': 'https://t.co/dTqiiMyFdj', 'twitter': 'http://twitter.com/neiljshaw', 'linkedin': 'https://www.linkedin.com/in/neil-shaw-b45369a/'}

{'name': 'James Rodger', 'verified': True, 'interests': 'United Kingdom', 'title': 'Head of Trends — Birmingham Live', 'location': 'Birmingham', 'description': 'Regional Content Editor for @birmingham_live.', 'seen_in': ['Wales on Sunday', 'Rochdale Observer', 'Daily Star (UK)', 'Western Mail', 'MSN South Africa', 'Daily Mirror', 'Birmingham Live', 'MSN UK', 'Sunday M